In [4]:
# Run this code block just once after the kernel starts
# to change current directory to root

import os

os.chdir("..")
print(os.getcwd())

/home/tunlaton11/Desktop/image-to-maps-project


In [12]:
from dataset import NuScenesDataset
from models.fpn import FPN50

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from tqdm import tqdm
import re
import platform

from configs.config_utilities import load_config

In [13]:
from configs.config_utilities import load_config

config = load_config()

In [31]:
sample_tokens = ["e3d495d4ac534d54b321f50006683844"]

train_dataset = NuScenesDataset(
    nuscenes_dir=config.nuscenes_dir,
    nuscenes_version=config.nuscenes_version,
    image_size=None,
    label_dir=config.label_dir,
    sample_tokens=sample_tokens
    # scene_names=config.train_scenes,
    # transform=train_transform,
)

train_loader = DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    num_workers=config.num_workers,
    pin_memory=True,
    shuffle=True,
)

validate_dataset = NuScenesDataset(
    nuscenes_dir=config.nuscenes_dir,
    nuscenes_version=config.nuscenes_version,
    image_size=(200, 196),
    label_dir=config.label_dir,
    sample_tokens=sample_tokens
    # scene_names=config.val_scenes,
)
validate_loader = DataLoader(
    validate_dataset,
    batch_size=config.batch_size,
    num_workers=config.num_workers,
    pin_memory=True,
    shuffle=True,
)

--------------------------------------------------
Loading NuScenes version v1.0-mini ...
--------------------------------------------------
--------------------------------------------------
Loading NuScenes version v1.0-mini ...
--------------------------------------------------


In [15]:
this_device = platform.platform()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    device = "cuda"
elif re.search("arm64", this_device):
    # use Apple GPU
    device = "mps"
else:
    device = "cpu"

In [16]:
network = FPN50().to(device)

In [32]:
images, labels, masks = next(iter(train_loader))
images = images.to(device)
labels = labels.to(device)
masks = masks.to(device)

In [34]:
outputs = network(images)

/home/tunlaton11/miniconda3/envs/senior-project/lib/python3.7/site-packages/torch/nn/functional.py:3734: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


In [33]:
print(images.shape)

torch.Size([1, 3, 900, 1600])


In [35]:
for i in outputs:
    print(i.shape)

torch.Size([1, 256, 113, 200])
torch.Size([1, 256, 57, 100])
torch.Size([1, 256, 29, 50])
torch.Size([1, 256, 15, 25])
torch.Size([1, 256, 8, 13])
